# 0. 문제 정의

-------------
# Wage prediction

현대축구에서는 우수한 측면수비수를 보유한 팀이 세계를 지배한다. 

피파 게임 데이터를 이용하여 누가 과연 좋은 측면수비수일지 예측해보자.  

물론 평가기준은... **돈(Wage)**이다!!  
**(프로는 돈으로 말한다)**

- Target : `Wage`
- Metric : `mse`
- Player : only.. 측면수비.. `RB`, `RWB`, `LB`, `LWB`
- Dataset: `FIFA 19`
- Remark : 상위 1000명의 선수를 `train:test=7:3`의 비율로 나누어 실습

이번 실습에 사용할 데이터를 임의로 분리하여 저장한다.

----------------------
# 모듈

In [1]:
import os
import sys
import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd
from time import sleep

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
from scipy import stats

import sklearn

------------------
# 데이터 불러오기

In [ ]:
# 디렉토리 내 파일 확인
os.listdir()

In [15]:
# 데이터 불러오기
fifa = pd.read_csv('fifa19.csv', index_col=0)
fifa = fifa[fifa.Position!='GK'].reset_index(drop=True)

# 측면수비수의 데이터만 남기기
position = ['LB','LWB','RB','RWB']
fifa = fifa.query("Position in @position").reset_index(drop=True)

# 실습에 이용할 데이터만 분리하기
fifa =  fifa.loc[:,['Wage','Value','Name', 'Age','Nationality',
       'Overall', 'Club','Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From', 'Contract Valid Until',
       'Height', 'Weight',
       'Crossing',
       'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle']]

# 키/몸무게 전처리 (숫자로 변환)
fifa[['feet','inch']] = fifa['Height'].str.split('[^\d]', expand=True).astype('int')
fifa['Height'] = 30.48 * fifa['feet'] + 2.54 * fifa['inch']
fifa['Weight'] = fifa.Weight.str.replace('[^\d]','').astype('float')*0.4536

# Wage 전처리 (숫자로 변환)
fifa['Wage'] = fifa['Wage'].str.replace('[^.\d]','').astype('float')

# 데이터 확인
fifa.head()

,Wage,Value,Name,Age,Nationality,Overall,Club,Preferred Foot,International Reputation,Weak Foot,...,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,feet,inch
0,285.0,€43M,Marcelo,30,Brazil,88,Real Madrid,Left,4.0,4.0,...,85.0,85.0,82.0,59.0,86.0,71.0,85.0,86.0,5,9
1,250.0,€38M,Jordi Alba,29,Spain,87,FC Barcelona,Left,3.0,3.0,...,84.0,79.0,70.0,59.0,79.0,72.0,84.0,85.0,5,7
2,160.0,€36.5M,Alex Sandro,27,Brazil,86,Juventus,Left,3.0,3.0,...,82.0,70.0,70.0,60.0,82.0,81.0,84.0,84.0,5,11
3,175.0,€35M,Azpilicueta,28,Spain,86,Chelsea,Right,3.0,3.0,...,89.0,62.0,74.0,66.0,78.0,88.0,90.0,86.0,5,10
4,110.0,€38M,D. Alaba,26,Austria,85,FC Bayern München,Left,4.0,4.0,...,84.0,76.0,77.0,80.0,79.0,80.0,82.0,80.0,5,11


---------------------
# 데이터 저장

In [13]:
fifa.head()

,Wage,Value,Name,Age,Nationality,Overall,Club,Preferred Foot,International Reputation,Weak Foot,...,Interceptions,Positioning,Vision,Penalties,Composure,Marking,StandingTackle,SlidingTackle,feet,inch
0,285.0,€43M,Marcelo,30,Brazil,88,Real Madrid,Left,4.0,4.0,...,85.0,85.0,82.0,59.0,86.0,71.0,85.0,86.0,5,9
1,250.0,€38M,Jordi Alba,29,Spain,87,FC Barcelona,Left,3.0,3.0,...,84.0,79.0,70.0,59.0,79.0,72.0,84.0,85.0,5,7
2,160.0,€36.5M,Alex Sandro,27,Brazil,86,Juventus,Left,3.0,3.0,...,82.0,70.0,70.0,60.0,82.0,81.0,84.0,84.0,5,11
3,175.0,€35M,Azpilicueta,28,Spain,86,Chelsea,Right,3.0,3.0,...,89.0,62.0,74.0,66.0,78.0,88.0,90.0,86.0,5,10
4,110.0,€38M,D. Alaba,26,Austria,85,FC Bayern München,Left,4.0,4.0,...,84.0,76.0,77.0,80.0,79.0,80.0,82.0,80.0,5,11


In [23]:
# 엑셀파일로 저장
# csv로 저장하면 외국어 깨지는 현상 발생 - 이유 모르겠음..
fifa.to_excel('fifa_wage_prediction.xlsx', index=False)

-----------------
# X_train, X_test / y_train, y_test 분리

- Overall 능력치 순 상위 1000명의 data를 7:3 비율로 `train set`과 `test set`으로 분리

- `random_state=1`로 고정하여 `shuffle`

- `train set`을 이용하여 모델을 만들어보고, `test set`은 최종결과를 확인할 때만 사용

In [ ]:
from sklearn.model_selection import train_test_split

# 능력치 순 상위 n명만 분리
n = 1000

# 변수 분리(X,y)
X   = fifa.loc[:n,['Name', 'Age','Nationality','Overall', 'Club',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Jersey Number', 'Joined', 'Loaned From',
       'Height', 'Weight',
       'Crossing',
       'Finishing', 'HeadingAccuracy', 'ShortPassing', 'Volleys', 'Dribbling',
       'Curve', 'FKAccuracy', 'LongPassing', 'BallControl', 'Acceleration',
       'SprintSpeed', 'Agility', 'Reactions', 'Balance', 'ShotPower',
       'Jumping', 'Stamina', 'Strength', 'LongShots', 'Aggression',
       'Interceptions', 'Positioning', 'Vision', 'Penalties', 'Composure',
       'Marking', 'StandingTackle', 'SlidingTackle']]

y = fifa.loc[:n,['Wage']]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.3,
                                                    random_state=1,
                                                    shuffle=True)

--------------
- 앞으로의 실습은 저장한 데이터를 불러와서, `train/test` 분리 후 진행하자